https://deep-eye.tistory.com/46

In [14]:
# import necessary packages
import cvlib as cv
import cv2

In [15]:
# 인터뷰 영상 , 겁나잘됨
video_file = "../asset/input/mov/mov_interview.mp4"
# # 길거리 영상, 겁나 안됨 yolo5 로 해보자
# video_file = "../asset/input/mov/mov_street.mp4"

## 동영상 불러오기
cap = cv2.VideoCapture(video_file)

In [24]:
## 동영상 불러오기
cap = cv2.VideoCapture(video_file)

while (cap.isOpened()) :
    ret, frame = cap.read()

    # 얼굴 감지
    ## 후에 YOLOv5로 수정
    face, confidence = cv.detect_face(frame)

    # 모자이크 처리
    for idx, f in enumerate(face):
        (startX, startY) = f[0], f[1]
        (endX, endY) = f[2], f[3]
        '모자이크 효과 주기: 얼굴 부분을 줄였다가 다시 원크기로 복구시키면 모자이크처럼 됨.'
        face_region = frame[startY:endY, startX:endX]
        M = face_region.shape[0]
        N = face_region.shape[1]
        try :
            ## 성별 및 나이를 감지해주는 코드
            ## gender detection (성별 검출), 처음 실행 시 다운로드 필요
            (label, confidence) = cv.detect_gender(frame)
            
            ## 얼굴 주위에 성별에 따른 박스를 그려주는 코드
            if label == 'male' :
                label_color = (255, 0, 0)
                cv2.rectangle(frame, (startX, startY), (endX, endY), label_color, 5)
            else :
                label_color = (0, 0, 255)
                cv2.rectangle(frame, (startX, startY), (endX, endY), label_color, 5)

            # fx와 fy값을 얼마로 정하냐에 따라 모자이크의 크기가 달라짐(작을수록 커지고 0.1만되도 잘 보이니까 그 이하로 잡을 것)
            face_region = cv2.resize(face_region, None, fx=0.06, fy=0.06, interpolation=cv2.INTER_AREA)
            face_region = cv2.resize(face_region, (N, M), interpolation=cv2.INTER_AREA)
            frame[startY:endY, startX:endX] = face_region
        except:
            print("*"*100)
            print(M,N)
            pass

    if ret :
        cv2.imshow('frame', frame)
        cv2.waitKey(3) # 1000/waitkey = FPS

        if cv2.waitKey(1) & 0xFF == ord('q') : break
    else :
        print("Fail to read frame!")
        break

cap.release()
cv2.destroyAllWindows()